In [119]:
import pandas as pd
import re
movies=pd.read_csv('movies.csv')
movies_db=pd.read_csv('movies_db.csv')

### 1.数据清洗

In [120]:
movies_db=movies_db.drop(
    columns=['durations','votes']
)

In [121]:
movies_db['INFO'] = (
    movies_db['genres'].fillna('').astype(str) + ' ' +
    movies_db['countries'].fillna('').astype(str) + ' ' +
    movies_db['reviews'].fillna('').astype(str)
)

movies_db = movies_db.drop(columns=['genres', 'countries', 'reviews'])

In [122]:
movies_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   subject_id  1172 non-null   object
 1   title       1172 non-null   object
 2   year        1172 non-null   object
 3   rating      1168 non-null   object
 4   directors   1169 non-null   object
 5   INFO        1172 non-null   object
dtypes: object(6)
memory usage: 55.1+ KB


删除无用列并只保留评分大于6.5的电影

In [123]:
movies = movies.drop(
    columns=['COVER','IMDB_ID','MINS','OFFICIAL_SITE','RELEASE_DATE','SLUG','ACTOR_IDS','DIRECTOR_IDS','LANGUAGES','GENRES','ALIAS','ACTORS']
)
movies = movies[movies['DOUBAN_SCORE'] >= 6.5]

筛选评分人数大于5000的电影并降序排列

In [124]:
movies_new = movies[(movies['DOUBAN_VOTES'] >= 3000)].sort_values(by=['DOUBAN_SCORE','DOUBAN_VOTES'], ascending=[False,False])[['DIRECTORS','MOVIE_ID','NAME','DOUBAN_SCORE','STORYLINE','TAGS','REGIONS','YEAR']]

### 2.余弦相似度模型构建

In [125]:
movies_new['INFO'] = (
    movies['STORYLINE'].fillna('').astype(str) + " " +
    movies['TAGS'].fillna('').astype(str) + " " +
    movies['REGIONS'].fillna('').astype(str)
#    + " " +
#    movies['DIRECTORS'].fillna('').astype(str) + " " +
#    movies['ACTORS'].fillna('').astype(str) + " " +
#    movies['YEAR'].fillna('').astype(str)
)
movies_new = movies_new.drop(columns=['STORYLINE', 'TAGS', 'REGIONS'
                                      #, 'DIRECTORS', 'ACTORS', 'YEAR'
                                     ])

In [126]:
movies_db_renamed = movies_db.rename(columns={
    'subject_id': 'MOVIE_ID',
    'title': 'NAME',
    'year': 'YEAR',
    'rating': 'DOUBAN_SCORE',
    'directors': 'DIRECTORS'
})

movies_db_renamed = movies_db_renamed[['DIRECTORS', 'MOVIE_ID', 'NAME', 'DOUBAN_SCORE', 'YEAR', 'INFO']]

movies_new = pd.concat([movies_new, movies_db_renamed], ignore_index=True)
movies_new['NAME'] = movies_new['NAME'].apply(lambda x: ''.join(re.findall(r'[\u4e00-\u9fff]+', str(x))))
print(movies_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5058 entries, 0 to 5057
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DIRECTORS     4913 non-null   object
 1   MOVIE_ID      5058 non-null   object
 2   NAME          5058 non-null   object
 3   DOUBAN_SCORE  5054 non-null   object
 4   YEAR          5058 non-null   object
 5   INFO          5058 non-null   object
dtypes: object(6)
memory usage: 237.2+ KB
None


In [127]:
#添加中文分词库
import jieba
from sklearn.feature_extraction.text import CountVectorizer

In [128]:
# 中文分词器
def chinese_tokenizer(text):
    return jieba.lcut(str(text))
stopwords = [
    "的", "了", "在", "是", "我", "有", "和", "就", "不", "人", 
    "都", "一", "一个", "上", "也", "很", "到", "说", "要", "去",
    "你", "会", "着", "没有", "看", "好", "自己", "这", "那", 
    "为", "之", "对", "与", "而", "并", "等", "被", "及", "或",
    "但", "所以", "如果", "因为", "然后", "而且", "那么", "他们", 
    "我们", "你们", "它们", "什么", "哪个", "哪些", "哪里", "时候",
    "他", "她", "它", "咱们", "大家", "谁", "怎样", "怎么", "多少", "为什么",
    "这里", "那里", "这样", "那样", "这个", "那个", "这些", "那些",
    "地", "得", "所", "过", "吗", "呢", "吧", "啊", "呀", "嘛", "哇", "啦",
    "从", "自", "以", "向", "关于", "对于", "根据", "按照", "通过", "由于",
    "并且", "或者", "虽然", "即使", "尽管", "不管", "只要", "只有", "除非",
    "最", "太", "更", "非常", "十分", "特别", "极其", "比较", "稍微", "有点",
    "刚", "才", "正在", "已经", "曾经", "马上", "立刻", "永远", "一直", "总是",
    "常常", "经常", "往往", "不断", "偶尔", "又", "再", "还", "仅", "光",
    "能", "能够", "可以", "可能", "应该", "应当", "想", "愿意", "肯", "敢",
    "来", "去", "进", "出", "回", "起", "开",
    "些", "一些", "所有", "每个", "某个", "各种", "多个", "几个", "第一", "第二",
    "就是", "只是", "可是", "真是", "也是", "不是", "也是", "就是", "正是",
    "一样", "一般", "一点", "一起", "一直", "一下", "一些", "一种", "一次"
]
cv = CountVectorizer(
    max_features=20000,
    tokenizer=chinese_tokenizer,
    stop_words=stopwords,   # 中文停用词表（如果有就传 list）
    token_pattern=None      # 必须设 None，否则 tokenizer 会被覆盖
)

vector = cv.fit_transform(movies_new['INFO'].astype(str)).toarray()

In [129]:
vector.shape

(5058, 20000)

In [130]:
from sklearn.metrics.pairwise import cosine_similarity

In [131]:
similarity=cosine_similarity(vector)

In [132]:
similarity

array([[1.        , 0.3355573 , 0.38554124, ..., 0.61230449, 0.75231426,
        0.37247195],
       [0.3355573 , 1.        , 0.68798422, ..., 0.57328294, 0.51467251,
        0.63862426],
       [0.38554124, 0.68798422, 1.        , ..., 0.5642217 , 0.51608263,
        0.59317101],
       ...,
       [0.61230449, 0.57328294, 0.5642217 , ..., 1.        , 0.79589961,
        0.78340297],
       [0.75231426, 0.51467251, 0.51608263, ..., 0.79589961, 1.        ,
        0.72044831],
       [0.37247195, 0.63862426, 0.59317101, ..., 0.78340297, 0.72044831,
        1.        ]])

In [133]:
import numpy as np

def recommand(movie_name, topk=5):
    label_idx = movies_new.index[movies_new['NAME'] == movie_name]
    if len(label_idx) == 0:
        print("未找到该影片")
        return
    pos = movies_new.index.get_loc(label_idx[0])

    sims = similarity[pos]
    cand = np.argsort(-sims)
    cand = cand[cand != pos][:topk]

    recs = []
    for j in cand:
        recs.append({
            "电影名": movies_new.iloc[j]['NAME'],
            "豆瓣评分": movies_new.iloc[j]['DOUBAN_SCORE'],
        })
    df = pd.DataFrame(recs)
    return df

In [134]:
recommand("蓝白红三部曲之蓝")

,电影名,豆瓣评分
0,浮生一日,8.9
1,枕边书,6.7
2,哈尔的移动城堡動城,9.1
3,黑暗中的舞者,8.2
4,戏梦巴黎,8.1


## 输入为电影集，GUI版

In [28]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
import numpy as np
import pandas as pd

class MovieCollectionRecommendationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🎬 电影集合推荐系统")
        self.root.geometry("900x700")
        
        self.setup_ui()
    
    def setup_ui(self):
        # 标题
        title_label = tk.Label(
            self.root, 
            text="🎬 豆MINI智能电影集合推荐系统", 
            font=("Arial", 16, "bold")
        )
        title_label.pack(pady=20)
        
        # 输入区域
        input_frame = tk.Frame(self.root)
        input_frame.pack(pady=10, fill=tk.X, padx=20)
        
        # 电影集合输入
        tk.Label(input_frame, text="输入电影集合:", font=("Arial", 12)).grid(row=0, column=0, sticky="w", pady=5)
        
        input_help = tk.Label(input_frame, text="(多个电影用逗号、分号或换行分隔)", font=("Arial", 9), fg="gray")
        input_help.grid(row=0, column=1, sticky="w", pady=5)
        
        self.movies_text = scrolledtext.ScrolledText(input_frame, height=4, width=60, font=("Arial", 11))
        self.movies_text.grid(row=1, column=0, columnspan=3, sticky="ew", pady=5)
        
        # 推荐设置区域
        settings_frame = tk.Frame(self.root)
        settings_frame.pack(pady=10, fill=tk.X, padx=20)
        
        tk.Label(settings_frame, text="推荐数量:", font=("Arial", 12)).pack(side=tk.LEFT, padx=5)
        
        self.topk_var = tk.StringVar(value="10")
        topk_spinbox = tk.Spinbox(
            settings_frame, 
            from_=1, 
            to=20, 
            width=5, 
            textvariable=self.topk_var,
            font=("Arial", 12)
        )
        topk_spinbox.pack(side=tk.LEFT, padx=5)
        
        # 推荐策略选择
        tk.Label(settings_frame, text="推荐策略:", font=("Arial", 12)).pack(side=tk.LEFT, padx=(20,5))
        
        self.strategy_var = tk.StringVar(value="average")
        strategy_combo = ttk.Combobox(
            settings_frame,
            textvariable=self.strategy_var,
            values=["average", "maximum", "union"],
            state="readonly",
            width=10
        )
        strategy_combo.pack(side=tk.LEFT, padx=5)
        
        # 策略说明
        strategy_help = tk.Label(settings_frame, text="(平均相似度 | 最大相似度 | 并集相似度)", font=("Arial", 9), fg="gray")
        strategy_help.pack(side=tk.LEFT, padx=5)
        
        # 按钮区域
        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        
        recommend_btn = tk.Button(
            button_frame,
            text="开始推荐",
            command=self.recommend_movies,
            font=("Arial", 12, "bold"),
            bg='#3498db',
            fg='white',
            padx=30,
            pady=5
        )
        recommend_btn.pack(side=tk.LEFT, padx=10)
        
        clear_btn = tk.Button(
            button_frame,
            text="清空输入",
            command=self.clear_input,
            font=("Arial", 12),
            bg='#95a5a6',
            fg='white',
            padx=20,
            pady=5
        )
        clear_btn.pack(side=tk.LEFT, padx=10)
        
        # 结果显示区域
        result_frame = tk.Frame(self.root)
        result_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # 创建文本框用于显示结果
        self.result_text = scrolledtext.ScrolledText(
            result_frame,
            wrap=tk.WORD,
            font=("Arial", 11),
            padx=10,
            pady=10
        )
        self.result_text.pack(fill=tk.BOTH, expand=True)
        
        # 状态栏
        self.status_var = tk.StringVar(value="就绪 - 请输入电影集合")
        status_bar = tk.Label(
            self.root, 
            textvariable=self.status_var,
            relief=tk.SUNKEN, 
            anchor=tk.W,
            font=("Arial", 10),
            bg='#f0f0f0'
        )
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # 添加一些提示
        self.show_welcome_message()
    
    def show_welcome_message(self):
        welcome_text = """✨ 欢迎使用电影集合推荐系统 ✨

使用方法：
1. 在输入框中输入多个电影名称（用逗号、分号或换行分隔）
2. 选择推荐数量和策略
3. 点击「开始推荐」按钮

推荐策略说明：
• 平均相似度：计算与所有输入电影的平均相似度
• 最大相似度：取与任意输入电影的最大相似度  
• 并集相似度：要求与所有输入电影都有一定相似度

示例输入：
肖申克的救赎, 霸王别姬, 阿甘正传
或：
肖申克的救赎
霸王别姬
阿甘正传

提示：确保输入正确的电影名称
"""
        self.result_text.insert(tk.END, welcome_text)
        self.result_text.config(state=tk.DISABLED)
    
    def clear_input(self):
        """清空输入框"""
        self.movies_text.delete(1.0, tk.END)
        self.status_var.set("输入已清空")
    
    def parse_movie_input(self, input_text):
        """解析用户输入的电影集合"""
        # 替换各种分隔符为逗号
        input_text = input_text.replace(';', ',').replace('，', ',').replace('\n', ',')
        # 分割并清理
        movies = [movie.strip() for movie in input_text.split(',') if movie.strip()]
        return movies
    
    def recommend_movies(self):
        """基于电影集合进行推荐"""
        input_text = self.movies_text.get(1.0, tk.END).strip()
        if not input_text:
            messagebox.showwarning("输入错误", "请输入电影名称")
            return
        
        try:
            topk = int(self.topk_var.get())
            strategy = self.strategy_var.get()
        except ValueError:
            messagebox.showwarning("输入错误", "推荐数量必须是数字")
            return
        
        # 解析输入的电影集合
        input_movies = self.parse_movie_input(input_text)
        if not input_movies:
            messagebox.showwarning("输入错误", "未找到有效的电影名称")
            return
        
        self.status_var.set(f"正在处理 {len(input_movies)} 部电影...")
        self.root.update()
        
        # 清空之前的结果
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete(1.0, tk.END)
        
        # 调用推荐函数
        try:
            result_df = self.get_collection_recommendations(input_movies, topk, strategy)
            
            if result_df is None:
                self.result_text.insert(tk.END, f"❌ 推荐失败\n\n")
                self.result_text.insert(tk.END, "💡 提示：请检查电影名称是否正确")
                self.status_var.set("推荐失败")
            else:
                # 显示推荐结果
                self.display_collection_recommendations(input_movies, result_df, topk, strategy)
                self.status_var.set(f"推荐完成 - 找到 {len(result_df)} 部推荐电影")
                
        except Exception as e:
            self.result_text.insert(tk.END, f"❌ 发生错误: {str(e)}")
            self.status_var.set("发生错误")
        
        self.result_text.config(state=tk.DISABLED)
    
    def get_collection_recommendations(self, movie_names, topk=10, strategy='average'):
        """基于电影集合的推荐函数"""
        try:
            # 验证输入电影
            valid_movies = []
            invalid_movies = []
            input_indices = []
            
            for movie in movie_names:
                movie_matches = movies_new[movies_new['NAME'] == movie]
                if len(movie_matches) > 0:
                    valid_movies.append(movie)
                    movie_idx = movie_matches.index[0]
                    pos = list(movies_new.index).index(movie_idx)
                    input_indices.append(pos)
                else:
                    invalid_movies.append(movie)
            
            if not valid_movies:
                return None
            
            # 记录无效电影（用于显示）
            self.invalid_movies = invalid_movies
            
            # 根据策略计算综合相似度
            if strategy == 'average':
                # 平均相似度
                combined_similarity = np.mean([similarity[i] for i in input_indices], axis=0)
            elif strategy == 'maximum':
                # 最大相似度
                combined_similarity = np.max([similarity[i] for i in input_indices], axis=0)
            elif strategy == 'union':
                # 并集相似度（最小相似度）
                combined_similarity = np.min([similarity[i] for i in input_indices], axis=0)
            else:
                combined_similarity = np.mean([similarity[i] for i in input_indices], axis=0)
            
            # 排序和过滤
            sim_scores = list(enumerate(combined_similarity))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            
            # 排除输入电影本身
            sim_scores = [(i, score) for i, score in sim_scores if i not in input_indices]
            
            if not sim_scores:
                return None
            
            # 取前topk个
            top_indices = [i for i, score in sim_scores[:topk]]
            top_scores = [score for i, score in sim_scores[:topk]]
            
            # 构建结果
            recs = []
            for j, sim_score in zip(top_indices, top_scores):
                movie_data = movies_new.iloc[j]
                rec = {
                    "电影名": movie_data['NAME'],
                    "豆瓣评分": movie_data['DOUBAN_SCORE'],
                    "综合相似度": sim_score
                }
                
                # 计算与每个输入电影的单独相似度
               # individual_sims = [similarity[input_idx][j] for input_idx in input_indices]
               # rec["各电影相似度"] = individual_sims
                
                # 添加可选信息
                if 'DIRECTORS' in movies_new.columns:
                    rec["导演"] = movie_data.get('DIRECTORS', '未知')
                if 'YEAR' in movies_new.columns:
                    rec["年份"] = movie_data.get('YEAR', '未知')
                
                recs.append(rec)
            
            return pd.DataFrame(recs)
            
        except Exception as e:
            print(f"推荐过程中出错: {e}")
            return None
    
    def display_collection_recommendations(self, input_movies, result_df, topk, strategy):
        """显示电影集合的推荐结果"""
        # 策略名称映射
        strategy_names = {
            'average': '平均相似度',
            'maximum': '最大相似度', 
            'union': '并集相似度'
        }
        
        strategy_name = strategy_names.get(strategy, strategy)
        
        # 构建输出文本
        output = f"""
🎬 电影集合推荐结果
{'='*60}
📚 输入电影: {', '.join(input_movies)}
🎯 推荐策略: {strategy_name}
📊 推荐数量: {topk} 部
{'='*60}

"""
        
        # 显示无效电影警告
        if hasattr(self, 'invalid_movies') and self.invalid_movies:
            output += f"⚠️  未找到以下电影: {', '.join(self.invalid_movies)}\n\n"
        
        # 显示推荐结果
        for i, row in result_df.iterrows():
            similarity_score = row['综合相似度']
            
            # 根据相似度选择符号
            if similarity_score > 0.7:
                sim_symbol = "【高相似】"
            elif similarity_score > 0.5:
                sim_symbol = "【中相似】" 
            elif similarity_score > 0.3:
                sim_symbol = "【低相似】"
            else:
                sim_symbol = "【一般】"
            
            output += f"{i+1:2d}. {sim_symbol} 《{row['电影名']}》\n"
            output += f"    ⭐ 评分: {row['豆瓣评分']} | 📊 综合相似度: {similarity_score:.3f}\n"
            
            # 显示与每个输入电影的相似度
            if '各电影相似度' in row:
                individual_sims = row['各电影相似度']
                sim_details = " | ".join([f"{sim:.3f}" for sim in individual_sims])
                output += f"    🔍 各电影相似度: {sim_details}\n"
            
            if '导演' in row and row['导演'] != '未知':
                output += f"    🎥 导演: {row['导演']}\n"
            if '年份' in row and row['年份'] != '未知':
                output += f"    📅 年份: {row['年份']}\n"
            
            if i < len(result_df) - 1:
                output += "    " + "─" * 50 + "\n"
        
        output += f"""
{'='*60}
✅ 推荐完成！基于 {len(input_movies)} 部输入电影，找到 {len(result_df)} 部推荐电影
📈 平均相似度: {result_df['综合相似度'].mean():.3f}
"""
        
        self.result_text.insert(tk.END, output)

# 启动应用程序
def start_collection_gui():
    root = tk.Tk()
    app = MovieCollectionRecommendationApp(root)
    root.mainloop()

# 运行GUI
if __name__ == "__main__":
    # 确保这些变量已经定义
    if 'movies_new' not in globals() or 'similarity' not in globals():
        print("错误: 请先运行数据准备代码")
    else:
        start_collection_gui()